# Деревья решений

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

In [ ]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    if example[int(feature_name)] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    if not isinstance(answer, dict):
        return answer
    
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * entropy(data_below) 
                      + p_data_above * entropy(data_above))
    
    return overall_entropy

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values >  split_value]
    
    return data_below, data_above

def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1): 
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)

        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification
    else:    
        counter += 1

        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        sub_tree = {question: []}
        
        Left = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        Right = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        if Left == Right:
            sub_tree = Left
        else:
            sub_tree[question].append(Left)
            sub_tree[question].append(Right)
        
        return sub_tree

def fit(train, target, max_depth = 5, min_samples = 2):
        data_frame = pd.DataFrame(train)
        data_frame['label'] = target
        tree = decision_tree_algorithm(data_frame, max_depth = max_depth, min_samples=min_samples)
        return tree
        
def predict(data,tree):
  tests = pd.DataFrame(data)
  return tests.apply(classify_example, axis=1, args=(tree,))

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(iris.data, iris.target, train_size=0.7)
tree = fit(train_data, train_target, max_depth = 10, min_samples = 0)
res = predict(test_data, tree)

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

sklearn_model = DecisionTreeClassifier()
sklearn_model.fit(train_data, train_target)
sklearn_prediction = sklearn_model.predict(test_data)

print("sklearn tree accuracy:", accuracy_score(sklearn_prediction, test_target))
print("My tree accuracy:", accuracy_score(res.values, test_target))

sklearn tree accuracy: 0.9555555555555556
My tree accuracy: 0.9555555555555556


Мое дерево по точности совпало с деревом из склерна. Это хорошо.
Реализованные критерии остановки: 
Purity, максимальная глубина, минимальное количество обьектов для сплита.

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import sklearn
import random
from pprint import pprint

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
import io
labelencoder = LabelEncoder()
churn = pd.read_csv(io.BytesIO(uploaded['churn.csv']))
churn_target = churn["Exited"]
churn["Gender"] = labelencoder.fit_transform(churn['Gender'])
churn["Geography"] = labelencoder.fit_transform(churn['Geography'])
churn_data = churn.drop(["RowNumber", "Surname", "CustomerId", "Exited"], axis = 1)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64
9996,516,0,1,35,10,57369.61,1,1,1,101699.77
9997,709,0,0,36,7,0.00,1,0,1,42085.58
9998,772,1,1,42,3,75075.31,2,1,0,92888.52


In [ ]:
class RFC:
  def __init__(self, max_tree_depth = 5, min_samples_leaf  = 5, tree_amount = 100):
        self.tree_amount = tree_amount
        self.max_tree_depth = max_tree_depth
        self.min_samples_leaf = min_samples_leaf
        self.trees = []
  def fit(self, X, Y):
    n = len(X)
    for i in range (0, self.tree_amount):
      X_train, Y_train = self.bootstrapping(X, Y, n)
      tree = DecisionTreeClassifier(max_features="auto", max_depth=self.max_tree_depth, min_samples_leaf=self.min_samples_leaf)
      tree.fit(X_train, Y_train)
      self.trees.append(tree)
  
  def predict(self, X):
    votes = []
    for tree in self.trees:
      votes.append(tree.predict(X))
    votes = pd.DataFrame(data = votes)
    return round(votes.mean())

  def feature_importances(self):
    votes = []
    for tree in self.trees:
      votes.append(tree.feature_importances_)
    votes = pd.DataFrame(data = votes)
    return votes.mean()
    
  def bootstrapping(self, train_df, train_df_target, n):
      seed = np.random.randint(1, 1000)
      X_sample = train_df.sample(n=n, random_state=seed, replace = True)
      Y_sample = train_df_target.sample(n=n, random_state=seed, replace = True)
      return X_sample, Y_sample


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(churn_data, churn_target, test_size = 0.33, random_state = 248)
myforest = RFC()
display(myforest.max_tree_depth)
myforest.fit( X_train, y_train)
predicted = myforest.predict(X_test)
accuracy_score(y_test, predicted)

5

0.8515151515151516

In [ ]:
def get_best_param():
  best_score = 0
  best_max_tree = 0
  best_max_depth = 0 
  best_min_samples_leaf = 0 
  for max_tree in [100, 200,500,1000]:
    for max_depth in [None,1,2,3,4,5]:
      for min_samples_leaf in range(1, 5):
        mF = RFC( min_samples_leaf, max_tree)
        mF.fit(X_train, y_train)
        y_pred = mF.predict(X_test)
        current_score = accuracy_score(y_test, y_pred)
        if current_score > best_score:
          best_score = current_score
          best_max_tree = max_tree
          best_max_depth = max_depth 
          best_min_samples_leaf = min_samples_leaf
  return best_max_depth, best_max_tree, best_min_samples_leaf, best_score

In [ ]:

display(get_best_param())

(None, 200, 4, 0.8345454545454546)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(churn_data, churn_target, test_size = 0.33, random_state = 3)
myforest = RFC(None, 4, 200)
myforest.fit( X_train, y_train)
predicted = myforest.predict(X_test)
accuracy_score(y_test, predicted)

0.8578787878787879

In [ ]:
from sklearn.ensemble import RandomForestClassifier
skForest = RandomForestClassifier(200,min_samples_leaf=4,max_depth=None)
skForest.fit(X_train, y_train)

predicted = skForest.predict(X_test)
accuracy_score(y_test, predicted)

0.8566666666666667

In [ ]:
display(pd.DataFrame(index=churn_data.columns, data=myforest.feature_importances().values, columns=['importance']).sort_values(by='importance', ascending=False))
display(pd.DataFrame(index=churn_data.columns, data=skForest.feature_importances_, columns=['importance']).sort_values(by='importance', ascending=False))

,importance
Age,0.302227
NumOfProducts,0.203313
Balance,0.118514
CreditScore,0.100822
EstimatedSalary,0.094915
IsActiveMember,0.056565
Tenure,0.051804
Geography,0.042350
Gender,0.018710
HasCrCard,0.010779


,importance
Age,0.302551
NumOfProducts,0.187981
Balance,0.118091
CreditScore,0.101715
EstimatedSalary,0.100162
IsActiveMember,0.057758
Tenure,0.057288
Geography,0.042186
Gender,0.020340
HasCrCard,0.011929


# New Section

**Вывод:**
Сравнив по метрике точности accuracy_score, можно сказать, что моя модель даже лучше чем sklearn, а значит лес написан правильно. Самым важным признаком является Age. 

Лучшими гиперпараметрами оказались:
количество деревьев = 200
максимальной глубины дерева = None
минимального числа объектов в листе = 4